# Tarea 2

Trabajaré con el conjunto de datos "incidentesviales_noviembre22". Al seguir investigando, descubrí que puedo utilizar la API de OpenStreetMap para complementar la información del camino, incluyendo el límite de velocidad, el número de carriles y el tipo de camino.

En lugar de predecir el número de accidentes por colonia, cambiaré mi enfoque a estimar la probabilidad de que ocurra un accidente en una cierta calle a una cierta hora, basándome en características meteorológicas y características del camino.

Mis objetivos para este tarea son 
1. Remover valores nulos de la columna 'hora'.
2. Corregir el formato de la columna 'fecha'.
3. Truncar la columna 'hora' para remover los minutos (debido a que el clima lo puedo obtener por hora) y agregarlo a la columna 'fecha'
4. Carga de los datos meteorológicos usando la API de Open Meteo y la informacion del camino con la API OpenStreetMap
5. Agrupar los incidentes por calle indicando el numero total de accidentes por calle, temperatura promedio y velocidad maxima permitida en la calle.

## Carga de los datos de incidentes viales
Dado que no utilizaré todas las columnas del archivo, solo cargaré las necesarias para conectar mis datos complementarios. 'fecha' y 'hora' son esenciales para los datos climatológicos, 'folio' servirá como identificador único para cada accidente, y 'longitud' y 'latitud' serán utilizados para obtener información acerca del camino.

In [37]:
import pandas as pd

# Nombre del archivo CSV
archivo_csv = "incidentesviales_noviembre22.csv"

# Especifica las columnas que deseas cargar
columnas_a_cargar = ['fecha', 'hora', 'folio', 'longitud', 'latitud']

# Lee el archivo CSV con solo las columnas especificadas
df_vial = pd.read_csv("../../Datos/" + archivo_csv, usecols=columnas_a_cargar)

# Imprime información básica sobre el DataFrame
print(df_vial.info())
df_vial

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167781 entries, 0 to 167780
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   fecha     167781 non-null  object 
 1   hora      167119 non-null  object 
 2   folio     167781 non-null  int64  
 3   longitud  167781 non-null  float64
 4   latitud   167781 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 6.4+ MB
None


,fecha,hora,folio,longitud,latitud
0,01/01/17,0:49,32858,-100.347430,25.740149
1,01/01/17,23:02,32857,-100.407075,25.739637
2,01/01/17,21:48,34349,-100.292420,25.651501
3,01/01/17,8:22,34888,-100.371026,25.728203
4,01/01/17,6:08,34310,-100.387760,25.753216
...,...,...,...,...,...
167776,30/11/22,23:15,103524,-100.352719,25.696954
167777,30/11/22,12:20,103368,-100.307290,25.685117
167778,30/11/22,7:05,103333,-100.273213,25.666169
167779,30/11/22,19:30,103597,-100.374777,25.737189


## Eliminación de valores nulos en la columna 'hora'
Al revisar la estructura del dataframe, observamos que la columna 'hora' contiene 167,119 valores no nulos de un total de 167,781 registros (lo que representa el 0.39% de nuestros datos). Por lo tanto, procedemos a eliminar las filas con valores nulos en la columna 'hora', ya que sin la hora no podemos asignarle un valor meteorológico.

In [2]:
# Elimina las filas donde la columna 'hora' es nula
df_vial.dropna(subset=['hora'], inplace= True)

# Imprime información actualizada del DataFrame
print(df_vial.info())

<class 'pandas.core.frame.DataFrame'>
Index: 167119 entries, 0 to 167780
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   fecha     167119 non-null  object 
 1   hora      167119 non-null  object 
 2   folio     167119 non-null  int64  
 3   longitud  167119 non-null  float64
 4   latitud   167119 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 7.7+ MB
None


## Corregir el formato de la columna 'fecha'
En este paso, abordaremos dos cuestiones relacionadas con la columna 'fecha':

* Detectamos inconsistencias donde en algunos registros el año está representado con 2 dígitos y en otros con 4, por ejemplo, "1/1/2017" y "30/11/22".
* La columna está registrada como una cadena de texto en lugar de estar en formato datetime.

Para solucionar esto, definiremos una función que realizará una división basada en "/", y cuando el elemento del año tenga solo 2 dígitos, agregará "20" para convertirlo en 4 dígitos. Luego, utilizaremos la función pd.to_datetime para convertir la columna al formato datetime.

> **Nota**  
> Como alternativa, podríamos utilizar un ciclo for para iterar a través de los registros y realizar el cambio utilizando df.at[i, 'fecha'], pero demostró ser más lento que el enfoque basado en una función.
>``` python
>for idx, row in df_vial.iterrows():
>    partes_fecha = row['fecha'].split("/")
>    if len(partes_fecha[2]) == 2:  # Si el año tiene solo dos dígitos
>        partes_fecha[2] = "20" + partes_fecha[2]
>    # Actualiza el valor en la fila 'fecha' en el DataFrame
>    df_vial.at[idx, 'fecha'] = "/".join(partes_fecha)
>```

In [38]:
# Define la función para corregir el formato de 'fecha'
def fix_date_format(date_str):
    partes_fecha = date_str.split("/")
    # Si el año tiene solo 2 dígitos, se asume que está en formato YY y se convierte a YYYY
    if len(partes_fecha[2]) == 2:
        partes_fecha[2] = "20" + partes_fecha[2]
    return "/".join(partes_fecha)

# Aplica la función 'fix_date_format' y convierte 'fecha' en datetime
df_vial['fecha'] = pd.to_datetime(df_vial['fecha'].apply(fix_date_format), format='%d/%m/%Y')

df_vial

,fecha,hora,folio,longitud,latitud
0,2017-01-01,0:49,32858,-100.347430,25.740149
1,2017-01-01,23:02,32857,-100.407075,25.739637
2,2017-01-01,21:48,34349,-100.292420,25.651501
3,2017-01-01,8:22,34888,-100.371026,25.728203
4,2017-01-01,6:08,34310,-100.387760,25.753216
...,...,...,...,...,...
167776,2022-11-30,23:15,103524,-100.352719,25.696954
167777,2022-11-30,12:20,103368,-100.307290,25.685117
167778,2022-11-30,7:05,103333,-100.273213,25.666169
167779,2022-11-30,19:30,103597,-100.374777,25.737189


In [39]:
# Combinamos 'fecha' y 'hora' en una nueva columna datetime
df_vial['fecha'] = pd.to_datetime(df_vial['fecha'].astype(str) + ' ' + pd.to_datetime(df_vial['hora'], format='%H:%M').dt.strftime('%H:00:00'))

# Eliminamos la columna 'hora' para evitar duplicación de información
df_vial.drop('hora', axis=1, inplace= True)

df_vial.head()

,fecha,folio,longitud,latitud
0,2017-01-01 00:00:00,32858,-100.347430,25.740149
1,2017-01-01 23:00:00,32857,-100.407075,25.739637
2,2017-01-01 21:00:00,34349,-100.292420,25.651501
3,2017-01-01 08:00:00,34888,-100.371026,25.728203
4,2017-01-01 06:00:00,34310,-100.387760,25.753216


## Carga de los datos meteorologicos
Para estos datos haremos uso del sitio [Open Meteo](https://open-meteo.com/) el cual provee una API de acceso gratuito para obtener datos historicos del clima por hora especificando latitud y longitud 
En este caso usaremos una direccion estatica ya que si lo hago dinamico y busco informacion para cada ubicacion exederia el limite de llamadas diarias a la API por lo tanto usare latitud = 25.68802 longitud = -100.31482 como referencia al municipio de monterrey 


In [6]:
import requests

# Definimos la URL de la API con los parámetros necesarios
api_url = "https://archive-api.open-meteo.com/v1/archive"
latitud = 25.68802
longitud = -100.31482
fecha_inicio = "2017-01-01"
fecha_fin = "2022-11-30"
variables_hora = "temperature_2m,relativehumidity_2m,precipitation,rain,weathercode,windspeed_10m,winddirection_10m,direct_radiation_instant"
zona_horaria = "America/Denver"

# Definimos los parámetros de la consulta
parametros = {
    "latitude": latitud,
    "longitude": longitud,
    "start_date": fecha_inicio,
    "end_date": fecha_fin,
    "hourly": variables_hora,
    "timezone": zona_horaria
}

# Realizamos una solicitud GET a la API
respuesta = requests.get(api_url, params=parametros)

# Verificamos si la solicitud fue exitosa (código de estado 200)
if respuesta.status_code == 200:
    datos = respuesta.json()
    
    df_clima = pd.DataFrame(datos["hourly"])
    
    # Conviertimos la columna 'time' al formato que trabajamos antes
    df_clima['time'] = pd.to_datetime(df_clima['time'], format='%Y-%m-%dT%H:%M').dt.strftime('%Y-%m-%d %H:%M:%S')
    df_clima['time'] = pd.to_datetime(df_clima['time'], format='%Y-%m-%d %H:%M:%S')
    # Imprime el DataFrame actualizado
    print(df_clima.head())

else:
    print("Error: No se pudo obtener datos de la API")


                 time  temperature_2m  relativehumidity_2m  precipitation  \
0 2017-01-01 00:00:00            17.8                   78            0.0   
1 2017-01-01 01:00:00            16.8                   81            0.0   
2 2017-01-01 02:00:00            16.6                   83            0.0   
3 2017-01-01 03:00:00            16.4                   83            0.0   
4 2017-01-01 04:00:00            15.9                   84            0.0   

   rain  weathercode  windspeed_10m  winddirection_10m  \
0   0.0            2            5.7                235   
1   0.0            2            6.3                246   
2   0.0            2            5.9                259   
3   0.0            2            6.7                254   
4   0.0            2            6.0                245   

   direct_radiation_instant  
0                       0.0  
1                       0.0  
2                       0.0  
3                       0.0  
4                       0.0  


## Hago un merge de los datos metereologicos
Hago un left join para agregar informacion del clima a mi tabla de accidentes uso la fecha y hora como identificador
y como ejercicio filtro para ver solo aquellos registros que sucedieran cuando el clima es mayor a 25 grados Celcius

In [15]:
df_new = df_vial.merge(df_clima, left_on= 'fecha', right_on= 'time', how = 'left').drop(columns= 'time')
df_new = df_new.head(500)
#df_new = df_new[df_new['temperature_2m']>=39]
#df_new[df_new['temperature_2m']>=39]

In [16]:
import overpy

# Crear una instancia de Overpass una vez para reutilizarla en todas las consultas
api = overpy.Overpass()

# Define la función para obtener información de la carretera
def get_road_info(lat, lon):
    # Crear la consulta
    query = f"""
        [out:json];
        (
          way(around:1000, {lat}, {lon})["highway"]["name"]["maxspeed"]["lanes"]["highway"];
        );
        out center;
    """
    
    # Enviar la consulta y obtener el resultado
    result = api.query(query)
    
    # Inicializar variables
    road_name, road_type, lanes, maxspeed = "N/A", "N/A", "N/A", "N/A"
    
    # Procesar el resultado para obtener la información de la carretera
    if result.ways:
        way = result.ways[0]  # Seleccionar la primera carretera encontrada (la más cercana)
        road_name = way.tags.get("name", "N/A")
        road_type = way.tags.get("highway", "N/A")
        lanes = way.tags.get("lanes", "N/A")
        maxspeed = way.tags.get("maxspeed", "N/A")
    
    return road_name, road_type, lanes, maxspeed

# Crear una lista de resultados para df_test
results = df_new.apply(lambda row: get_road_info(row['latitud'], row['longitud']), axis=1)
# Convertir la lista de resultados en un DataFrame con columnas separadas
result_df = pd.DataFrame(results.tolist(), columns=['road_name', 'road_type', 'lanes', 'maxspeed'])

# Concatenar el DataFrame resultante con df_test
df_new = pd.concat([df_new, result_df], axis=1)
df_new
#print(df_new)

,fecha,folio,longitud,latitud,temperature_2m,relativehumidity_2m,precipitation,rain,weathercode,windspeed_10m,winddirection_10m,direct_radiation_instant,road_name,road_type,lanes,maxspeed
0,2017-01-01 00:00:00,32858,-100.347430,25.740149,17.8,78,0.0,0.0,2,5.7,235,0.0,N/A,N/A,N/A,N/A
1,2017-01-01 23:00:00,32857,-100.407075,25.739637,18.3,74,0.0,0.0,0,2.7,203,0.0,Avenida Cumbres Élite,tertiary,2,40
2,2017-01-01 21:00:00,34349,-100.292420,25.651501,20.9,71,0.0,0.0,0,6.0,155,0.0,Avenida Fernando García Roel,tertiary,2,30
3,2017-01-01 08:00:00,34888,-100.371026,25.728203,17.3,72,0.0,0.0,1,2.3,231,7.0,Avenida Adolfo Ruíz Cortines,primary,3,40
4,2017-01-01 06:00:00,34310,-100.387760,25.753216,15.9,85,0.0,0.0,2,6.0,245,0.0,Avenida Adolfo Ruíz Cortines,primary,3,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017-01-09 08:00:00,35164,-100.287903,25.644369,9.1,58,0.0,0.0,0,2.3,252,14.1,Avenida Fernando García Roel,tertiary,2,30
496,2017-01-09 12:00:00,35454,-100.348503,25.734226,17.5,46,0.0,0.0,0,5.4,98,607.8,Avenida Licenciado Boulevard Raúl Rangel Frías,primary,2,60
497,2017-01-09 08:00:00,35451,-100.362550,25.699851,9.1,58,0.0,0.0,0,2.3,252,14.1,Avenida Paseo de los Leones,primary,2,80
498,2017-01-09 10:00:00,35452,-100.371474,25.701504,13.5,52,0.0,0.0,0,2.1,59,360.0,Avenida Paseo de los Leones,primary,3,80
